In [1]:
from dataclasses import dataclass
from agents import function_tool, RunContextWrapper
from agents import Agent, Runner
from pydantic import BaseModel
import asyncio

# Basic Tool usage

In [ ]:
@dataclass
class MyCtx:
    user_id: str
    call_count: int = 0

    def __str__(self):
        print(f"user_id: {self.user_id}, call_count: {self.call_count}")


In [ ]:
@function_tool(name_override="increment")
def increment_counter(
    ctx: RunContextWrapper[MyCtx], 
    note: str
) -> str:
    
    print(f"increment_counter called: {ctx}")
    
    ctx.context.call_count += 1
    return (
        f"User {ctx.context.user_id} 增加计数，当前 = {ctx.context.call_count}；"
        f"备注: {note}"
    )


In [ ]:
agent = Agent(
    name="Counter Agent",
    instructions="Call the `increment` tool whenever the user asks.",
    tools=[increment_counter],
)

In [ ]:
ctx_obj = MyCtx(user_id="alice")

res1 = await Runner.run(
    agent,
    'Please increment with note "first hit"',
    context=ctx_obj 
)
print("→", res1.final_output)
print("Ctx after 1st:", ctx_obj)

# Pass agent by funciton tool

In [3]:
class Ctx(BaseModel):
    user_id: str

@function_tool(name_override="show_agent_info")
def show_agent_info(
    ctx: RunContextWrapper[Ctx], 
    extra: str
) -> str:
    agent = ctx.agent
    agent.instructions += f"\n\n(tagged: {extra})"
    return (
        f"I'm agent: {agent.name}\n"
        f"My tools: {[t.name for t in agent.tools]}\n"
        f"Extra: {extra}"
    )

In [4]:
root = Agent(
    name="Root",
    instructions="Root agent — prints info.",
    tools=[show_agent_info],
)

In [7]:
res = await Runner.run(
        root,
        'Call show_agent_info with {"extra": "v1"}',
        context=Ctx(user_id="alice")
    )

In [8]:
res.final_output

'It seems there was an error when attempting to retrieve the information. Would you like me to try again or assist you with something else?'